In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import metrics
from sklearn import preprocessing

from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, GridSearchCV, ShuffleSplit

from matplotlib.pyplot import plot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

%matplotlib inline

Считываем данные:

In [2]:
train = pd.read_csv('Desktop/ML/data/Task5/train.csv')
train.columns = ['id', 'author', 'content', 'sentiment']
train.head()

,id,author,content,sentiment
0,26898,richardepryor,"@treasaint salad stuff, some chillis, whatever...",happiness
1,27635,reese,"@sunnyjamiel sunny, I'm a workin' on it. It's ...",neutral
2,3036,mutedriposte,@jolynnchew so early??,surprise
3,5604,sakizzie_1102,"So now, I have conjunctivitis in my left eye. ...",sadness
4,36111,poptrash,"Out and about in Deal, Kent. More sunshine req...",love


In [3]:
test = pd.read_csv('Desktop/ML/data/Task5/test.csv')
test.columns = ['id', 'author', 'content']
test.head()

,id,author,content
0,32823,valicast,Good Morning
1,16298,btb103,I just put my computer up on craigslist. I've ...
2,28505,anavil,in ten minutes shopping demi lovato-back aro...
3,6689,ritwik1st,From twitterberry moved to ubertwitter - suffe...
4,26893,TightFreebies,@thriftymom TEAR*


In [4]:
print(train.shape)
print(test.shape)

(30000, 4)
(10000, 3)


In [5]:
train.drop('id', axis = 1, inplace = True)
test.drop('id', axis = 1, inplace = True)

In [6]:
train.drop('author', axis = 1, inplace = True)
test.drop('author', axis = 1, inplace = True)

In [7]:
print(train.shape)
print(test.shape)

(30000, 2)
(10000, 1)


In [8]:
train.head()

,content,sentiment
0,"@treasaint salad stuff, some chillis, whatever...",happiness
1,"@sunnyjamiel sunny, I'm a workin' on it. It's ...",neutral
2,@jolynnchew so early??,surprise
3,"So now, I have conjunctivitis in my left eye. ...",sadness
4,"Out and about in Deal, Kent. More sunshine req...",love


In [9]:
test.head()

,content
0,Good Morning
1,I just put my computer up on craigslist. I've ...
2,in ten minutes shopping demi lovato-back aro...
3,From twitterberry moved to ubertwitter - suffe...
4,@thriftymom TEAR*


In [10]:
y_train = train["sentiment"]
X_train = train.drop('sentiment', axis=1, inplace=False)
X_test = test 

In [11]:
print(X_train.shape)
print(X_test.shape)

(30000, 1)
(10000, 1)


In [12]:
count_vectorizer = CountVectorizer()

In [13]:
sparse_feature_matrix = count_vectorizer.fit_transform(list(X_train.values.ravel()))

Подберем параметры:

In [14]:
C_var = [0.1 * i for i in range(1, 9, 2)]
max_iter_var = range(10, 310, 100)
estimators = []

for c in C_var:
    for m in max_iter_var:
        estimators.append(LogisticRegression(penalty='l1', dual=False, tol=0.0001, C=c, fit_intercept=True,
                                             intercept_scaling=1, class_weight=None, random_state=None,
                                             solver='liblinear', max_iter=m, multi_class='ovr', 
                                             verbose=0, warm_start=False, n_jobs=1))

In [15]:
answer1 = []
answer2 = []
ans_estimator = []

cv_strategy = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
for estimator in estimators:
    scoring = cross_val_score(estimator, sparse_feature_matrix, y_train, cv=5, scoring='accuracy')
    answer1.append(scoring.mean())
    answer2.append(scoring.std())
    ans_estimator.append(estimator)

In [16]:
print(answer1[answer1.index(max(answer1))])

0.35556503313367893


In [17]:
indexes = []

for i in range(len(answer1)):
    if answer1[i] > 0.35:
        indexes.append(i)

minimum = 1
ind_min = None
for ind in indexes:
    if answer2[ind] < minimum:
        minimum = answer2[ind]
        ind_min = ind
        
print(ind_min)
print(answer2[ind_min])
print(answer1[ind_min])

10
0.004481537385357563
0.35229864264612376


In [18]:
ans_estimator[ind_min]

LogisticRegression(C=0.7000000000000001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=110,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [19]:
algo = ans_estimator[ind_min]

Запускаем на выбранных параметрах:

In [20]:
algo.fit(sparse_feature_matrix, y_train)

LogisticRegression(C=0.7000000000000001, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=110,
          multi_class='ovr', n_jobs=1, penalty='l1', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [21]:
preds = algo.predict(count_vectorizer.transform(list(X_test.values.ravel())))

In [22]:
def submit(preds):
    submission = pd.read_csv("Desktop/ML/data/Task5/sample_submission.csv")
    submission["sentiment"] = preds
    submission.to_csv("Desktop/ML/data/Task5/submission12.csv", index=False)

In [23]:
submit(preds)

In [24]:
!head Desktop/ML/data/Task5/submission12.csv

id,sentiment
32823,neutral
16298,neutral
28505,worry
6689,neutral
26893,neutral
36572,happiness
12335,worry
29591,happiness
18948,worry
